In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


In [83]:
output = torch.randint(0, 51, (5,1,128,128,3))/50 # B, 1, H, W, C (mask, T2NO, T2ND-T2NO)
gt = torch.randint(0, 51, (5,1,128,128,2))/50 # B, 1, H, W, C (T2NO, T2ND-T2NO)

In [91]:
def masked_mse(output, gt):
    gt_mask = (gt[...,0]!=1).float()
    bce = F.binary_cross_entropy_with_logits(output[...,0],gt_mask)

    mask = gt_mask.bool()

    if mask.sum() != 0:
        masked_t2no_pred, masked_t2nd_pred  = output[...,1][mask], output[...,2][mask]
        masked_t2no_gt, masked_t2nd_gt      = gt[...,0][mask], gt[...,1][mask]

        mse_t2no = F.mse_loss(masked_t2no_pred, masked_t2no_gt)
        mse_t2nd = F.mse_loss(masked_t2nd_pred, masked_t2nd_gt)
    else:
        mse_t2no = torch.tensor(0.0, device=output.device)
        mse_t2nd = torch.tensor(0.0, device=output.device)

    return bce + mse_t2no + mse_t2nd

# out = torch.tensor(np.array([(gt[...,0]!=1).float(), gt[...,0], gt[...,1]])).permute(1,2,3,4,0)
masked_mse(output, gt)

tensor(0.8374)